# Data Exploration and Enrichment

### Load the datasets

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load main data
data_df = pd.read_csv("/Users/elbethelzewdie/Downloads/ethiopia-fi-forecast/ethiopia-fi-forecast/data/raw/ethiopia_fi_unified_data.csv")

# Load impact links
impact_links_df = pd.read_csv("/Users/elbethelzewdie/Downloads/ethiopia-fi-forecast/ethiopia-fi-forecast/data/raw/impact_sheet.csv")

# Load reference codes
ref_codes_df = pd.read_csv("/Users/elbethelzewdie/Downloads/ethiopia-fi-forecast/ethiopia-fi-forecast/data/raw/reference_codes.csv")

### Explore the Schema (All Records Share Same Columns)

In [3]:
print("Data shape:", data_df.shape)
print("\nColumns:")
print(data_df.columns)

print("\nData types:")
print(data_df.dtypes)

print("\nSample rows:")
data_df.head()


Data shape: (43, 34)

Columns:
Index(['record_id', 'record_type', 'category', 'pillar', 'indicator',
       'indicator_code', 'indicator_direction', 'value_numeric', 'value_text',
       'value_type', 'unit', 'observation_date', 'period_start', 'period_end',
       'fiscal_year', 'gender', 'location', 'region', 'source_name',
       'source_type', 'source_url', 'confidence', 'related_indicator',
       'relationship_type', 'impact_direction', 'impact_magnitude',
       'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country',
       'collected_by', 'collection_date', 'original_text', 'notes'],
      dtype='object')

Data types:
record_id               object
record_type             object
category                object
pillar                  object
indicator               object
indicator_code          object
indicator_direction     object
value_numeric          float64
value_text              object
value_type              object
unit                    object
observat

,record_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,...,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,REC_0001,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,22.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Baseline year,NaN
1,REC_0002,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,35.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
2,REC_0003,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,46.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,NaN,NaN
3,REC_0004,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,56.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN
4,REC_0005,observation,NaN,ACCESS,Account Ownership Rate,ACC_OWNERSHIP,higher_better,36.0,NaN,percentage,...,NaN,NaN,NaN,NaN,NaN,Example_Trainee,2025-01-20,NaN,Gender disaggregated,NaN


### Identify Record Types (Observation / Event / Target)

In [4]:
data_df['record_type'].value_counts()

record_type
observation    30
event          10
target          3
Name: count, dtype: int64

In [5]:
observations_df = data_df[data_df['record_type'] == 'observation']
events_df = data_df[data_df['record_type'] == 'event']
targets_df = data_df[data_df['record_type'] == 'target']


In [6]:
print("Observations:", len(observations_df))
print("Events:", len(events_df))
print("Targets:", len(targets_df))


Observations: 30
Events: 10
Targets: 3


### Understand Observations

In [7]:
observations_df[
    ['indicator_code', 'value_numeric', 'observation_date', 'fiscal_year']
].head()

,indicator_code,value_numeric,observation_date,fiscal_year
0,ACC_OWNERSHIP,22.0,2014-12-31,2014
1,ACC_OWNERSHIP,35.0,2017-12-31,2017
2,ACC_OWNERSHIP,46.0,2021-12-31,2021
3,ACC_OWNERSHIP,56.0,2021-12-31,2021
4,ACC_OWNERSHIP,36.0,2021-12-31,2021


In [8]:
acc_by_gender = (
    observations_df[
        observations_df['indicator_code'] == 'ACC_OWNERSHIP'
    ]
    .groupby(['fiscal_year', 'gender'])['value_numeric']
    .mean()
    .reset_index()
)


### Understand Events

In [9]:
EVENT_ID = 'record_id'
EVENT_TYPE = 'category'
EVENT_DATE = 'observation_date'

In [10]:
events_df[[EVENT_ID, EVENT_TYPE, 'pillar', EVENT_DATE]].head()


,record_id,category,pillar,observation_date
33,EVT_0001,product_launch,NaN,2021-05-17
34,EVT_0002,market_entry,NaN,2022-08-01
35,EVT_0003,product_launch,NaN,2023-08-01
36,EVT_0004,infrastructure,NaN,2024-01-01
37,EVT_0005,policy,NaN,2024-07-29


In [11]:
events_df['category'].value_counts()

category
product_launch    2
infrastructure    2
policy            2
market_entry      1
milestone         1
partnership       1
pricing           1
Name: count, dtype: int64

In [12]:
events_df['pillar'].isna().mean()

np.float64(1.0)

### Review Reference Codes (Validation)

In [13]:
print(ref_codes_df.columns.tolist())

['field', 'code', 'description', 'applies_to']


In [14]:
ref_codes_df['field'].value_counts()

field
value_type             11
category               10
source_type             8
pillar                  7
record_type             6
confidence              4
relationship_type       4
impact_direction        4
impact_magnitude        4
evidence_basis          4
indicator_direction     3
gender                  3
location                3
Name: count, dtype: int64

In [15]:
valid_indicators = ref_codes_df[
    ref_codes_df['field'] == 'indicator_code'
]['code'].unique()

invalid_obs = observations_df[
    ~observations_df['indicator_code'].isin(valid_indicators)
]

print("Invalid indicator records:", len(invalid_obs))


Invalid indicator records: 30


In [16]:
valid_event_categories = ref_codes_df[
    ref_codes_df['field'] == 'category'
]['code'].unique()

invalid_events = events_df[
    ~events_df['category'].isin(valid_event_categories)
]

print("Invalid event categories:", len(invalid_events))


Invalid event categories: 0


In [17]:
valid_pillars = ref_codes_df[
    ref_codes_df['field'] == 'pillar'
]['code'].unique()

invalid_pillars = observations_df[
    ~observations_df['pillar'].isin(valid_pillars)
]

print("Invalid pillar entries:", len(invalid_pillars))


Invalid pillar entries: 0


In [18]:
print(observations_df[~observations_df['indicator_code'].isin(valid_indicators)][['indicator_code']].drop_duplicates())


        indicator_code
0        ACC_OWNERSHIP
6       ACC_MM_ACCOUNT
8           ACC_4G_COV
10      ACC_MOBILE_PEN
11           ACC_FAYDA
14       USG_P2P_COUNT
16       USG_P2P_VALUE
17       USG_ATM_COUNT
18       USG_ATM_VALUE
19       USG_CROSSOVER
20  USG_TELEBIRR_USERS
21  USG_TELEBIRR_VALUE
22     USG_MPESA_USERS
23    USG_MPESA_ACTIVE
24     USG_ACTIVE_RATE
25     AFF_DATA_INCOME
26         GEN_GAP_ACC
28        GEN_MM_SHARE
29      GEN_GAP_MOBILE


### Link Events to Impacted Indicators via related_indicator

In [19]:
print(impact_links_df.columns.tolist())

['record_id', 'parent_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']


In [20]:
event_impacts = events_df.merge(
    impact_links_df,
    left_on='record_id',
    right_on='parent_id',
    how='left',
    suffixes=('_event', '_impact')
)


In [21]:
event_indicator_obs = event_impacts.merge(
    observations_df[['indicator_code', 'pillar']],
    left_on='related_indicator_impact',   # use the impact suffix
    right_on='indicator_code',
    how='left'
)


In [22]:
print(event_impacts.columns.tolist())

['record_id_event', 'record_type_event', 'category_event', 'pillar_event', 'indicator_event', 'indicator_code_event', 'indicator_direction_event', 'value_numeric_event', 'value_text_event', 'value_type_event', 'unit_event', 'observation_date_event', 'period_start_event', 'period_end_event', 'fiscal_year_event', 'gender_event', 'location_event', 'region_event', 'source_name_event', 'source_type_event', 'source_url_event', 'confidence_event', 'related_indicator_event', 'relationship_type_event', 'impact_direction_event', 'impact_magnitude_event', 'impact_estimate_event', 'lag_months_event', 'evidence_basis_event', 'comparable_country_event', 'collected_by_event', 'collection_date_event', 'original_text_event', 'notes_event', 'record_id_impact', 'parent_id', 'record_type_impact', 'category_impact', 'pillar_impact', 'indicator_impact', 'indicator_code_impact', 'indicator_direction_impact', 'value_numeric_impact', 'value_text_impact', 'value_type_impact', 'unit_impact', 'observation_date_im

In [23]:
print(event_indicator_obs[['record_id_event', 'related_indicator_impact', 'indicator_code', 'pillar']].head())

  record_id_event related_indicator_impact indicator_code  pillar
0        EVT_0001            ACC_OWNERSHIP  ACC_OWNERSHIP  ACCESS
1        EVT_0001            ACC_OWNERSHIP  ACC_OWNERSHIP  ACCESS
2        EVT_0001            ACC_OWNERSHIP  ACC_OWNERSHIP  ACCESS
3        EVT_0001            ACC_OWNERSHIP  ACC_OWNERSHIP  ACCESS
4        EVT_0001            ACC_OWNERSHIP  ACC_OWNERSHIP  ACCESS


In [24]:
event_pillars = (
    event_indicator_obs
    .groupby('record_id_event')['pillar']  # or 'pillar' with suffix if needed
    .apply(lambda x: list(x.dropna().unique()))
    .reset_index(name='inferred_pillars')
)
print(event_pillars.head())


  record_id_event         inferred_pillars
0        EVT_0001          [ACCESS, USAGE]
1        EVT_0002  [ACCESS, AFFORDABILITY]
2        EVT_0003          [USAGE, ACCESS]
3        EVT_0004         [ACCESS, GENDER]
4        EVT_0005          [AFFORDABILITY]


In [25]:
for _, event in event_impacts.iterrows():
    event_date = pd.to_datetime(event['observation_date_event'])
    indicator = event['related_indicator_impact']

    indicator_obs = observations_df[
        (observations_df['indicator_code'] == indicator) &
        (observations_df['value_numeric'].notna())
    ].copy()

    indicator_obs['observation_date'] = pd.to_datetime(indicator_obs['observation_date'])

    before = indicator_obs[indicator_obs['observation_date'] < event_date]['value_numeric']
    after = indicator_obs[indicator_obs['observation_date'] >= event_date]['value_numeric']

    print(f"Event {event['record_id_event']} Impact on {indicator}:")
    print(f" Before mean: {before.mean()}, After mean: {after.mean()}")


Event EVT_0001 Impact on ACC_OWNERSHIP:
 Before mean: 28.5, After mean: 46.75
Event EVT_0001 Impact on USG_TELEBIRR_USERS:
 Before mean: nan, After mean: 54840000.0
Event EVT_0001 Impact on USG_P2P_COUNT:
 Before mean: nan, After mean: 89000000.0
Event EVT_0002 Impact on ACC_4G_COV:
 Before mean: nan, After mean: 54.15
Event EVT_0002 Impact on AFF_DATA_INCOME:
 Before mean: nan, After mean: 2.0
Event EVT_0003 Impact on USG_MPESA_USERS:
 Before mean: nan, After mean: 10800000.0
Event EVT_0003 Impact on ACC_MM_ACCOUNT:
 Before mean: 4.7, After mean: 9.45
Event EVT_0004 Impact on ACC_OWNERSHIP:
 Before mean: 39.0, After mean: 49.0
Event EVT_0004 Impact on GEN_GAP_ACC:
 Before mean: 20.0, After mean: 18.0
Event EVT_0005 Impact on AFF_DATA_INCOME:
 Before mean: nan, After mean: 2.0
Event EVT_0006 Impact on nan:
 Before mean: nan, After mean: nan
Event EVT_0007 Impact on USG_MPESA_ACTIVE:
 Before mean: 7100000.0, After mean: nan
Event EVT_0007 Impact on USG_P2P_COUNT:
 Before mean: 89000000.

In [27]:
impact_summary = []

for _, event in event_impacts.iterrows():
    event_date = pd.to_datetime(event['observation_date_event'])
    lag = event['lag_months_impact']
    if pd.notna(lag):
        event_date = event_date + pd.DateOffset(months=int(lag))

    indicator = event['related_indicator_impact']

    indicator_obs = observations_df[
        (observations_df['indicator_code'] == indicator) &
        (observations_df['value_numeric'].notna())
    ].copy()
    indicator_obs['observation_date'] = pd.to_datetime(indicator_obs['observation_date'])

    before = indicator_obs[indicator_obs['observation_date'] < event_date]['value_numeric']
    after = indicator_obs[indicator_obs['observation_date'] >= event_date]['value_numeric']

    impact_summary.append({
        'event_id': event['record_id_event'],
        'indicator': indicator,
        'lag_months': lag,
        'before_mean': before.mean(),
        'after_mean': after.mean(),
        'before_count': before.count(),
        'after_count': after.count()
    })

impact_df = pd.DataFrame(impact_summary)

print(impact_df.head())


   event_id           indicator  lag_months  before_mean  after_mean  \
0  EVT_0001       ACC_OWNERSHIP        12.0         39.0        49.0   
1  EVT_0001  USG_TELEBIRR_USERS         3.0          NaN  54840000.0   
2  EVT_0001       USG_P2P_COUNT         6.0          NaN  89000000.0   
3  EVT_0002          ACC_4G_COV        12.0         37.5        70.8   
4  EVT_0002     AFF_DATA_INCOME        12.0          NaN         2.0   

   before_count  after_count  
0             5            1  
1             0            1  
2             0            2  
3             1            1  
4             0            1  


### Count records by record_type, pillar, source_type, and confidence

In [28]:
print("Record counts by record_type:")
print(data_df['record_type'].value_counts(), "\n")

print("Record counts by pillar:")
print(data_df['pillar'].value_counts(dropna=False), "\n")

print("Record counts by source_type:")
print(data_df['source_type'].value_counts(dropna=False), "\n")

print("Record counts by confidence:")
print(data_df['confidence'].value_counts(dropna=False), "\n")


Record counts by record_type:
record_type
observation    30
event          10
target          3
Name: count, dtype: int64 

Record counts by pillar:
pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64 

Record counts by source_type:
source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64 

Record counts by confidence:
confidence
high      40
medium     3
Name: count, dtype: int64 



### Identify the temporal range of observations

In [29]:
observations_df.loc[:, 'observation_date'] = pd.to_datetime(observations_df['observation_date'], errors='coerce')
min_date = observations_df['observation_date'].min()
max_date = observations_df['observation_date'].max()

print(f"Temporal range of observations: {min_date.date()} to {max_date.date()}")


Temporal range of observations: 2014-12-31 to 2025-12-31


### List all unique indicators (indicator_code) and their coverage (number of observations per indicator)

In [30]:
indicator_counts = observations_df['indicator_code'].value_counts()
print("Unique indicators and their observation counts:")
print(indicator_counts)


Unique indicators and their observation counts:
indicator_code
ACC_OWNERSHIP         6
ACC_FAYDA             3
ACC_4G_COV            2
USG_P2P_COUNT         2
GEN_GAP_ACC           2
ACC_MM_ACCOUNT        2
USG_MPESA_USERS       1
GEN_MM_SHARE          1
AFF_DATA_INCOME       1
USG_ACTIVE_RATE       1
USG_MPESA_ACTIVE      1
USG_CROSSOVER         1
USG_TELEBIRR_VALUE    1
USG_TELEBIRR_USERS    1
USG_ATM_VALUE         1
USG_ATM_COUNT         1
USG_P2P_VALUE         1
ACC_MOBILE_PEN        1
GEN_GAP_MOBILE        1
Name: count, dtype: int64


### Understand which events are cataloged and their dates

In [31]:
events_df.loc[:, 'observation_date'] = pd.to_datetime(events_df['observation_date'], errors='coerce')

print("Event categories and their counts:")
print(events_df['category'].value_counts())

print("\nTemporal range of events:")
print(f"From {events_df['observation_date'].min().date()} to {events_df['observation_date'].max().date()}")


Event categories and their counts:
category
product_launch    2
infrastructure    2
policy            2
market_entry      1
milestone         1
partnership       1
pricing           1
Name: count, dtype: int64

Temporal range of events:
From 2021-05-17 to 2025-12-18


### Review the existing impact_links and what relationships they capture

In [32]:
print("Impact links columns and sample:")
print(impact_links_df.columns.tolist())
print(impact_links_df.head())

print("\nUnique relationship types in impact_links:")
print(impact_links_df['relationship_type'].value_counts(dropna=False))


Impact links columns and sample:
['record_id', 'parent_id', 'record_type', 'category', 'pillar', 'indicator', 'indicator_code', 'indicator_direction', 'value_numeric', 'value_text', 'value_type', 'unit', 'observation_date', 'period_start', 'period_end', 'fiscal_year', 'gender', 'location', 'region', 'source_name', 'source_type', 'source_url', 'confidence', 'related_indicator', 'relationship_type', 'impact_direction', 'impact_magnitude', 'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country', 'collected_by', 'collection_date', 'original_text', 'notes']
  record_id parent_id  record_type  category         pillar  \
0  IMP_0001  EVT_0001  impact_link       NaN         ACCESS   
1  IMP_0002  EVT_0001  impact_link       NaN          USAGE   
2  IMP_0003  EVT_0001  impact_link       NaN          USAGE   
3  IMP_0004  EVT_0002  impact_link       NaN         ACCESS   
4  IMP_0005  EVT_0002  impact_link       NaN  AFFORDABILITY   

                                indicator  ind